In [1]:
%load_ext autoreload
%autoreload 2


In [36]:

import sys
if '../' not in sys.path: 
    sys.path.append('../')

from src.normalizer import *
from src.space_groups import *

from itertools import product


from src.srdegrees import SR_Degrees
prepare_gap_env()

In [32]:
[range(int(D[i,i])) for i in range(r)]

[range(0, 1), range(0, 2)]

In [38]:
D, U, V = A.smith_form()
U * A * V

[1 0]
[0 2]

In [41]:
def cosets_lattice(A): 
    if A.det() == 0: 
        raise NotImplementedError()
        
    D, U, V = A.smith_form()
    n = D.rank() 
    gens = [list(range(D[i, i])) for i in range(n)]
    cosets = []
    for v in product(*gens): 
        v = vector(QQ, v)
        cosets.append(V.inverse() * v)
    return cosets

In [58]:
import random 

for _ in range(100):
    A = matrix(QQ, [
        [random.randint(-5, 5) for _ in range(2)] 
        for _ in range(2)
    ])
    if A.det() != 0: 
        break 

cosets_lattice(A)

[(0, 0)]

In [63]:
??A.smith_form

Docstring:
   Return a Smith normal form of this matrix.

   For a matrix M, a Smith normal form is a matrix S = UMV such that:

   * U and V are invertible matrices

   * the only non-vanishing entries of S are located on the diagonal
     (though S might not be a square matrix)

   * if d_i denotes the entry of S at (i,i), then d_i divides d_{i+1}
     for all i, i.e., the d_i are the ordered "elementary_divisors()"
     of M

   Note that the matrices U and V are not uniquely determined and the
   d_i are only uniquely determined up to units. For some base rings,
   such as local rings, the d_i might be further normalized, see
   "LOCAL RINGS" below.

   If the base ring is not a PID, the routine might work, or else it
   will fail having found an example of a non-principal ideal. Note
   that we do not call any methods to check whether or not the base
   ring is a PID, since this might be quite expensive (e.g. for rings
   of integers of number fields of large degree).

   INPUT:



In [62]:
A

[-1 -2]
[-1 -4]

In [60]:
A.det()

2

In [51]:
D, U, V = A.smith_form()
D

[1 0 0]
[0 1 0]
[0 0 1]

In [52]:
U * A * V

[1 0 0]
[0 1 0]
[0 0 1]

In [10]:
SR_Degrees(7).algorithm()

=========================== Group 7 ==================================
Generators of group:

[ [-1  0  0]  [ -1   0 1/2]  [1 0 1]  [1 0 0] ]
[ [ 0 -1  0]  [  0   1   0]  [0 1 0]  [0 1 1] ]
[ [ 0  0  1], [  0   0   1], [0 0 1], [0 0 1] ]

SNoT

[ [1 0 0]  [1 0 0]  [-1  0  0]  [ -1   0 1/2]  [   1    0 -1/2] ]
[ [0 1 0]  [0 1 0]  [ 0 -1  0]  [  0   1   0]  [   0   -1    0] ]
[ [0 0 1], [0 0 1], [ 0  0  1], [  0   0   1], [   0    0    1] ]


[ [x0  0]  [ 0 x0] ]
[ [ 0 x1], [x1  0] ]


----------------Dilation-------------------------

... testing inverse A (should have integral entities):

[x0  0]
[ 0 x1]

                           [1 0|0]
      [1 0 0]    [1 0 0]   [0 1|0]
a_inv [0 1 0]a = [0 1 0] = [---+-]
      [0 0 1]    [0 0 1]   [0 0|1]

                           [1 0|0]
      [1 0 0]    [1 0 0]   [0 1|0]
a_inv [0 1 0]a = [0 1 0] = [---+-]
      [0 0 1]    [0 0 1]   [0 0|1]

                                       [-1  0| 0]
      [-1  0  0]    [  -1    0 2*a0]   [ 0 -1| 0]
a_inv 

In [16]:
G, P, _, _ = build_group(7)
P

[
[1 0]  [-1  0]  [-1  0]  [ 1  0]
[0 1], [ 0 -1], [ 0  1], [ 0 -1]
]

In [33]:
PG = MatrixGroup(P).as_permutation_group()
PG

Permutation Group with generators [(), (1,2)(3,4), (1,2), (3,4)]

In [49]:
x = MatrixGroup(P)
x

Matrix group over Rational Field with 4 generators (
[1 0]  [-1  0]  [-1  0]  [ 1  0]
[0 1], [ 0 -1], [ 0  1], [ 0 -1]
)

In [76]:
normalizers(7, ignore_trivial=False)

[
[x0  0]  [ 0 x0]  [x1  0]  [ 0  0]  [ 0  0]  [ 0 x1]
[ 0  0], [ 0  0], [ 0 x0], [x0  0], [ 0 x0], [x0  0]
]

In [44]:

for el in gap(MatrixGroup(P)).AutomorphismGroup().AsList(): 
    print(el)

TypeError: gap() takes exactly 1 positional argument (2 given)